# Computer Vision Group Project

## Setup

### Installations

In [ ]:
!pip install opencv-python decord matplotlib numpy plotly
%matplotlib inline

### Libraries

In [ ]:
import os
import cv2
from decord import VideoReader, cpu
import matplotlib.pyplot as plt
import numpy as np
import time
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

## Frame Extractions and Pair Generation

In [ ]:
def extract_frames_with_decord(video_path, frames_dir, every=1, overwrite=False):
    os.makedirs(frames_dir, exist_ok=True)

    vr = VideoReader(video_path, ctx=cpu(0))
    total_frames = len(vr)
    saved = 0
    extracted_frame_indices = []  # Keep track of which frames we actually extracted

    for idx in range(0, total_frames, every):
        frame = vr[idx].asnumpy()
        frame_filename = os.path.join(frames_dir, f"frame_{idx:04d}.jpg")

        if not os.path.exists(frame_filename) or overwrite:
            cv2.imwrite(frame_filename, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
            saved += 1
            extracted_frame_indices.append(idx)

    print(f"{saved} frames saved to {frames_dir}")
    print(f"Extracted frame indices: {extracted_frame_indices[:10]}...")  # Show first 10
    return saved, extracted_frame_indices

video_path = "Vid6.mp4"         # path to input video
frames_output = "extracted_frames"  # folder to save frames

saved_count, EXTRACTED_FRAME_INDICES = extract_frames_with_decord(video_path, frames_output, every=5)

def generate_evaluation_pairs(extracted_indices, num_pairs=8):
    pairs = []
    total_frames = len(extracted_indices)
    
    # Half small gaps, half large gaps
    num_small = num_pairs // 2
    num_large = num_pairs - num_small
    
    # Small gaps (2 frames apart)
    step = max(1, total_frames // (num_small + 2))
    for i in range(0, total_frames - 2, step):
        if len(pairs) >= num_small:
            break
        idx1 = extracted_indices[i]
        idx2 = extracted_indices[i + 2]
        pairs.append((idx1, idx2))
    
    # Large gaps (spread across the video)
    for i in range(num_large):
        start_pos = (i * total_frames) // num_large
        end_pos = min(start_pos + total_frames // 4, total_frames - 1)  # Jump by ~25% of video
        if start_pos < end_pos:
            idx1 = extracted_indices[start_pos]
            idx2 = extracted_indices[end_pos]
            pairs.append((idx1, idx2))
    
    return pairs

frame_pairs = generate_evaluation_pairs(EXTRACTED_FRAME_INDICES, num_pairs=8)
print(f"Selected {len(frame_pairs)} frame pairs for qualitative assessment:")
for i, (idx1, idx2) in enumerate(frame_pairs):
    gap = abs(idx2 - idx1)
    print(f"  Pair {i+1}: ({idx1}, {idx2}) - gap: {gap} frames")

## Feature Detection and Description

### Loading algorithms

In [ ]:
def kps_descriptors(detector, label=""):
    keypoints = []
    descriptors = []

    frame_files = sorted([f for f in os.listdir(frames_output) if f.endswith(".jpg")])

    start_time = time.time()  # Start timing

    for filename in frame_files:
        frame_path = os.path.join(frames_output, filename)
        image = cv2.imread(frame_path, cv2.IMREAD_COLOR)
        if image is None:
            continue
        kps, desc = detector.detectAndCompute(image, None)
        keypoints.append(kps)
        descriptors.append(desc)

    end_time = time.time()  # End timing
    elapsed = end_time - start_time
    print(f"{label} took {elapsed:.2f} seconds to detect and describe {len(keypoints)} frames.")

    return keypoints, descriptors

orb = cv2.ORB_create(nfeatures=2000)
sift = cv2.SIFT_create()
akaze = cv2.AKAZE_create()

orb_kps, orb_desc = kps_descriptors(cv2.ORB_create(nfeatures=2000), label="ORB")
sift_kps, sift_desc = kps_descriptors(cv2.SIFT_create(), label="SIFT")
akaze_kps, akaze_desc = kps_descriptors(cv2.AKAZE_create(), label="AKAZE")

kps_dict = {
    "ORB": orb_kps,
    "SIFT": sift_kps,
    "AKAZE": akaze_kps
}
desc_dict = {
    "ORB": orb_desc,
    "SIFT": sift_desc,
    "AKAZE": akaze_desc
}


### Number of detected features

In [ ]:
def counts(detector_data):
    plt.figure(figsize=(10, 6))
    for label, keypoints in detector_data:
        frame_numbers = list(range(len(keypoints)))
        keypoint_counts = [len(kps) for kps in keypoints]
        plt.plot(frame_numbers, keypoint_counts, label=label, marker='o')

    plt.xlabel("Frame Number")
    plt.ylabel("Number of Keypoints Detected")
    plt.title("Keypoints per Frame by Detector")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

counts(list(kps_dict.items()))

### Feature Matching and Matching Performance

In [ ]:
def match_and_draw_combined(img1, kps1_dict, desc1_dict, img2, kps2_dict, desc2_dict, methods=["ORB", "SIFT", "AKAZE"]):
    
    fig, axes = plt.subplots(1, 3, figsize=(21, 6))
    
    for idx, method in enumerate(methods):
        kps1 = kps1_dict[method]
        desc1 = desc1_dict[method]
        kps2 = kps2_dict[method]
        desc2 = desc2_dict[method]
        
        if method == "ORB" or method == "AKAZE":
            bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
            matches = bf.match(desc1, desc2)
        elif method == "SIFT":
            bf = cv2.BFMatcher(cv2.NORM_L2)
            raw_matches = bf.knnMatch(desc1, desc2, k=2)
            matches = [m for m, n in raw_matches if m.distance < 0.75 * n.distance]
        
        matches = sorted(matches, key=lambda x: x.distance)
        
        matched_img = cv2.drawMatches(img1, kps1, img2, kps2, matches[:100], None, flags=2)
        
        axes[idx].imshow(cv2.cvtColor(matched_img, cv2.COLOR_BGR2RGB))
        axes[idx].set_title(f"{method} - {len(matches)} total matches")
        axes[idx].axis("off")
    
    plt.tight_layout()
    plt.show()
    
    return {method: matches for method, matches in zip(methods, [matches])}

def load_frame(idx):
    path = os.path.join(frames_output, f"frame_{idx:04d}.jpg")
    return cv2.imread(path, cv2.IMREAD_COLOR)

def compare_matching_performance(frame_pairs, kps_dict, desc_dict, methods=["ORB", "SIFT", "AKAZE"]):
    
    for idx1, idx2 in frame_pairs:
        img1 = load_frame(idx1)
        img2 = load_frame(idx2)

        if img1 is None or img2 is None:
            print(f"Skipped pair ({idx1}, {idx2}) due to missing frame.")
            continue

        # Convert frame indices to data structure indices
        try:
            i1 = EXTRACTED_FRAME_INDICES.index(idx1)
            i2 = EXTRACTED_FRAME_INDICES.index(idx2)
        except ValueError:
            print(f"Frame indices {idx1} or {idx2} not found in extracted frames.")
            continue

        print(f"\nMatching frames {idx1} and {idx2}:")

        # Create dictionaries for this frame pair
        kps1_dict = {method: kps_dict[method][i1] for method in methods}
        desc1_dict = {method: desc_dict[method][i1] for method in methods}
        kps2_dict = {method: kps_dict[method][i2] for method in methods}
        desc2_dict = {method: desc_dict[method][i2] for method in methods}
        
        match_and_draw_combined(img1, kps1_dict, desc1_dict, img2, kps2_dict, desc2_dict, methods)
        
compare_matching_performance(frame_pairs, kps_dict, desc_dict)

## Outlier Rejection

In [ ]:
def run_ransac_combined(img1, img2, kps1_dict, kps2_dict, matches_dict, methods=["ORB", "SIFT", "AKAZE"]):
    
    fig, axes = plt.subplots(1, 3, figsize=(21, 6))
    inliers_dict = {}
    
    for idx, method in enumerate(methods):
        matches = matches_dict[method]
        kps1 = kps1_dict[method]
        kps2 = kps2_dict[method]
        
        if len(matches) < 4:
            axes[idx].text(0.5, 0.5, f"Not enough matches\nfor RANSAC ({len(matches)} matches)", 
                          ha='center', va='center', transform=axes[idx].transAxes)
            axes[idx].set_title(f"{method} - RANSAC Failed")
            axes[idx].axis("off")
            inliers_dict[method] = []
            continue

        pts1 = np.float32([kps1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        pts2 = np.float32([kps2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

        H, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, 5.0)
        if mask is None:
            axes[idx].text(0.5, 0.5, "RANSAC failed to\ncompute homography", 
                          ha='center', va='center', transform=axes[idx].transAxes)
            axes[idx].set_title(f"{method} - RANSAC Failed")
            axes[idx].axis("off")
            inliers_dict[method] = []
            continue

        inliers = [m for i, m in enumerate(matches) if mask[i]]
        inliers_dict[method] = inliers

        ransac_img = cv2.drawMatches(img1, kps1, img2, kps2, inliers[:50], None, flags=2)
        
        axes[idx].imshow(cv2.cvtColor(ransac_img, cv2.COLOR_BGR2RGB))
        axes[idx].set_title(f"{method} - {len(inliers)} inliers (After RANSAC)")
        axes[idx].axis("off")
    
    plt.tight_layout()
    plt.show()
    
    return inliers_dict

def compare_matching_performance_ransac(frame_pairs, kps_dict, desc_dict, methods=["ORB", "SIFT", "AKAZE"]):
    
    for idx1, idx2 in frame_pairs:
        img1 = load_frame(idx1)
        img2 = load_frame(idx2)

        if img1 is None or img2 is None:
            print(f"Skipped pair ({idx1}, {idx2}) due to missing frame.")
            continue

        # Convert frame indices to data structure indices
        try:
            i1 = EXTRACTED_FRAME_INDICES.index(idx1)
            i2 = EXTRACTED_FRAME_INDICES.index(idx2)
        except ValueError:
            print(f"Frame indices {idx1} or {idx2} not found in extracted frames.")
            continue

        print(f"\nMatching frames {idx1} and {idx2}:")

        # Create dictionaries for this frame pair
        kps1_dict = {method: kps_dict[method][i1] for method in methods}
        desc1_dict = {method: desc_dict[method][i1] for method in methods}
        kps2_dict = {method: kps_dict[method][i2] for method in methods}
        desc2_dict = {method: desc_dict[method][i2] for method in methods}
        
        # Get matches for all methods
        matches_dict = {}
        for method in methods:
            if method == "ORB" or method == "AKAZE":
                bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                matches = bf.match(desc1_dict[method], desc2_dict[method])
            elif method == "SIFT":
                bf = cv2.BFMatcher(cv2.NORM_L2)
                raw_matches = bf.knnMatch(desc1_dict[method], desc2_dict[method], k=2)
                matches = [m for m, n in raw_matches if m.distance < 0.75 * n.distance]
            matches_dict[method] = sorted(matches, key=lambda x: x.distance)
        
        # Show initial matches
        match_and_draw_combined(img1, kps1_dict, desc1_dict, img2, kps2_dict, desc2_dict, methods)
        
        # Show RANSAC results
        run_ransac_combined(img1, img2, kps1_dict, kps2_dict, matches_dict, methods)

compare_matching_performance_ransac(frame_pairs, kps_dict, desc_dict)


## Fundamental Matrix Estimation and 3D Reconstruction Functions

In [ ]:
def estimate_fundamental_matrix(pts1, pts2):
    """Estimate fundamental matrix using RANSAC for uncalibrated cameras"""
    if len(pts1) < 8:
        print(f"Not enough points for fundamental matrix estimation ({len(pts1)} points).")
        return None, None
    
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC, 3.0, 0.99)
    
    if F is None:
        print("Failed to estimate fundamental matrix.")
        return None, None
    
    inlier_pts1 = pts1[mask.ravel() == 1]
    inlier_pts2 = pts2[mask.ravel() == 1]
    
    return F, (inlier_pts1, inlier_pts2)

def compute_camera_matrices_from_fundamental(F, pts1, pts2, img_shape):
    """
    Compute camera projection matrices from fundamental matrix for uncalibrated cameras.
    Uses canonical camera matrices approach.
    """
    h, w = img_shape[:2]
    
    # For uncalibrated cameras, we use canonical camera matrices
    # First camera at origin
    P1 = np.array([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0]], dtype=np.float32)
    
    # Compute epipoles from fundamental matrix
    # e2 is the left null vector of F (Fe2 = 0)
    U, S, Vt = np.linalg.svd(F)
    e2 = Vt[-1, :]  # Last row of Vt
    e2 = e2 / e2[2] if e2[2] != 0 else e2  # Normalize
    
    # Create skew-symmetric matrix from e2
    e2_skew = np.array([[0, -e2[2], e2[1]],
                        [e2[2], 0, -e2[0]],
                        [-e2[1], e2[0], 0]])
    
    # Second camera matrix: P2 = [e2_skew * F | e2]
    P2 = np.hstack([e2_skew @ F, e2.reshape(-1, 1)])
    
    return P1, P2

def triangulate_points_uncalibrated(F, pts1, pts2, img_shape):
    """
    Triangulate 3D points using fundamental matrix for uncalibrated cameras
    """
    # Get camera projection matrices from fundamental matrix
    P1, P2 = compute_camera_matrices_from_fundamental(F, pts1, pts2, img_shape)
    
    # Triangulate points using the projection matrices
    points_4d = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)
    
    # Convert from homogeneous to 3D coordinates
    points_3d = points_4d[:3] / points_4d[3]
    
    # Filter out points behind cameras (negative Z in first camera frame)
    valid_mask = points_4d[3] != 0  # Valid homogeneous coordinates
    points_3d_filtered = points_3d.T[valid_mask.flatten()]
    
    return points_3d_filtered, P1, P2

def process_frame_pair_3d(frame_idx1, frame_idx2, method="SIFT"):
    """Process a pair of frames and return 3D points using fundamental matrix"""
    
    # Load frames
    img1 = load_frame(frame_idx1)
    img2 = load_frame(frame_idx2)
    
    if img1 is None or img2 is None:
        return None, None, None
    
    # Convert frame indices to data structure indices using extracted frame indices
    try:
        i1 = EXTRACTED_FRAME_INDICES.index(frame_idx1)
        i2 = EXTRACTED_FRAME_INDICES.index(frame_idx2)
    except ValueError:
        print(f"Frame indices {frame_idx1} or {frame_idx2} not found in extracted frames.")
        return None, None, None
    
    # Get keypoints and descriptors
    kps1 = kps_dict[method][i1]
    desc1 = desc_dict[method][i1]
    kps2 = kps_dict[method][i2]
    desc2 = desc_dict[method][i2]
    
    if desc1 is None or desc2 is None:
        return None, None, None
    
    # Match features
    if method == "ORB" or method == "AKAZE":
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(desc1, desc2)
    elif method == "SIFT":
        bf = cv2.BFMatcher(cv2.NORM_L2)
        raw_matches = bf.knnMatch(desc1, desc2, k=2)
        matches = [m for m, n in raw_matches if m.distance < 0.75 * n.distance]
    
    if len(matches) < 10:
        return None, None, None
    
    # Extract corresponding points
    pts1 = np.float32([kps1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([kps2[m.trainIdx].pt for m in matches])
    
    # Estimate fundamental matrix (not essential matrix for uncalibrated case)
    F, inliers = estimate_fundamental_matrix(pts1, pts2)
    
    if F is None or inliers is None:
        return None, None, None
    
    inlier_pts1, inlier_pts2 = inliers
    
    # Triangulate 3D points using fundamental matrix
    points_3d, P1, P2 = triangulate_points_uncalibrated(F, inlier_pts1, inlier_pts2, img1.shape)
    
    # Filter out extreme outliers based on distance from origin
    if len(points_3d) > 0:
        distances = np.linalg.norm(points_3d, axis=1)
        # Remove points that are too far (likely reconstruction errors)
        valid_mask = distances < np.percentile(distances, 95)  # Keep 95% of closest points
        points_3d_filtered = points_3d[valid_mask]
    else:
        points_3d_filtered = points_3d
    
    return points_3d_filtered, F, (P1, P2)

## 3D Reconstruction for Multiple Frame Pairs

In [ ]:
def reconstruct_3d_scene(methods=["SIFT", "ORB", "AKAZE"], max_pairs=5):
    """Reconstruct 3D scene from multiple frame pairs"""
    
    # Generate reconstruction pairs from actual extracted frames
    # Use every 4th extracted frame to ensure good baseline while maintaining overlap
    reconstruction_indices = EXTRACTED_FRAME_INDICES[::4]  # Take every 4th extracted frame
    reconstruction_pairs = []
    
    for i in range(len(reconstruction_indices) - 1):
        if len(reconstruction_pairs) >= max_pairs:
            break
        frame1 = reconstruction_indices[i]
        frame2 = reconstruction_indices[i + 1]
        reconstruction_pairs.append((frame1, frame2))
    
    print(f"Using reconstruction pairs: {reconstruction_pairs}")
    
    results = {}
    
    for method in methods:
        print(f"\nProcessing with {method}...")
        method_results = []
        
        for frame1, frame2 in reconstruction_pairs:
            print(f"  Processing pair ({frame1}, {frame2})")
            
            points_3d, F, pose = process_frame_pair_3d(frame1, frame2, method)
            
            if points_3d is not None and len(points_3d) > 0:
                method_results.append({
                    'pair': (frame1, frame2),
                    'points_3d': points_3d,
                    'fundamental_matrix': F,
                    'pose': pose,
                    'num_points': len(points_3d)
                })
                print(f"    Reconstructed {len(points_3d)} 3D points")
            else:
                print(f"    Failed to reconstruct pair ({frame1}, {frame2})")
        
        results[method] = method_results
    
    return results

# Run 3D reconstruction
reconstruction_results = reconstruct_3d_scene()

# Display summary statistics
print("\n=== Reconstruction Summary ===")
for method, results in reconstruction_results.items():
    if results:
        total_points = sum([r['num_points'] for r in results])
        successful_pairs = len(results)
        print(f"{method}: {successful_pairs} successful pairs, {total_points} total 3D points")
    else:
        print(f"{method}: No successful reconstructions")

## 3D Visualisation

In [ ]:
import matplotlib.pyplot as plt

def create_interactive_3d_plot(reconstruction_results, max_points_per_method=500):
    """Create interactive 3D visualisation with Plotly"""
    
    methods_with_data = [(method, results) for method, results in reconstruction_results.items() if results]
    
    if not methods_with_data:
        print("No reconstruction data available for visualisation.")
        return
    
    n_methods = len(methods_with_data)
    
    # Create subplots
    subplot_titles = [method for method, _ in methods_with_data]
    fig = make_subplots(
        rows=1, cols=n_methods,
        specs=[[{'type': 'scatter3d'} for _ in range(n_methods)]],
        subplot_titles=subplot_titles
    )
    
    color_palette = ['red', 'blue', 'green', 'orange', 'purple']
    
    for idx, (method, results) in enumerate(methods_with_data):
        all_points = []
        colors = []
        
        for i, result in enumerate(results):
            points = result['points_3d']
            
            # Subsample points
            if len(points) > max_points_per_method // len(results):
                indices = np.random.choice(len(points), max_points_per_method // len(results), replace=False)
                points = points[indices]
            
            all_points.extend(points)
            colors.extend([color_palette[i % len(color_palette)]] * len(points))
        
        if all_points:
            all_points = np.array(all_points)
            
            # Remove extreme outliers
            for dim in range(3):
                coord = all_points[:, dim]
                q1, q3 = np.percentile(coord, [25, 75])
                iqr = q3 - q1
                lower_bound = q1 - 1.5 * iqr
                upper_bound = q3 + 1.5 * iqr
                valid_mask = (coord >= lower_bound) & (coord <= upper_bound)
                all_points = all_points[valid_mask]
                colors = [colors[i] for i in range(len(colors)) if valid_mask[i]]
            
            # Add scatter trace
            fig.add_trace(
                go.Scatter3d(
                    x=all_points[:, 0],
                    y=all_points[:, 1], 
                    z=all_points[:, 2],
                    mode='markers',
                    marker=dict(
                        size=2,
                        color=colors,
                        opacity=0.6
                    ),
                    showlegend=False
                ),
                row=1, col=idx+1
            )
    
    fig.update_layout(
        title_text="3D Reconstruction Results - Click and drag to rotate",
        height=500
    )
    
    # Update 3D scene properties for each subplot
    for i in range(n_methods):
        scene_key = f'scene{i+1}' if i > 0 else 'scene'
        fig.update_layout(**{
            scene_key: dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z',
                aspectmode='cube'
            )
        })
    
    fig.show()
    return fig

# Create the interactive visualization
print("Creating interactive 3D visualisation...")
print("Note: You can rotate by dragging and/or zoom by scrolling into the 3D plots.")
visualization_fig = create_interactive_3d_plot(reconstruction_results)

## Analysis and Comparison of Reconstruction Results

In [ ]:
def analyze_reconstruction_quality(reconstruction_results):
    """Analyze and compare reconstruction quality across methods"""
    
    print("=== Detailed Reconstruction Analysis ===\n")
    
    method_stats = {}
    
    for method, results in reconstruction_results.items():
        if not results:
            print(f"{method}: No successful reconstructions")
            continue
            
        point_counts = [r['num_points'] for r in results]
        total_points = sum(point_counts)
        avg_points = total_points / len(results) if results else 0
        successful_pairs = len(results)
        
        all_points = []
        for result in results:
            all_points.extend(result['points_3d'])
        
        if all_points:
            all_points = np.array(all_points)
            spreads = [np.std(all_points[:, i]) for i in range(3)]
            avg_spread = np.mean(spreads)
        else:
            avg_spread = 0
        
        method_stats[method] = {
            'successful_pairs': successful_pairs,
            'total_points': total_points,
            'avg_points_per_pair': avg_points,
            'avg_spread': avg_spread
        }
        
        print(f"{method}:")
        print(f"  Successful pairs: {successful_pairs}")
        print(f"  Total 3D points: {total_points}")
        print(f"  Average points per pair: {avg_points:.1f}")
        print(f"  Average coordinate spread: {avg_spread:.2f}")
        print()
    
    # Create comparison plot
    if method_stats:
        methods = list(method_stats.keys())
        total_points = [method_stats[m]['total_points'] for m in methods]
        successful_pairs = [method_stats[m]['successful_pairs'] for m in methods]
        
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Total Points by Method', 'Successful Reconstructions by Method')
        )
        
        fig.add_trace(
            go.Bar(x=methods, y=total_points, name='Total Points'),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Bar(x=methods, y=successful_pairs, name='Successful Pairs'),
            row=1, col=2
        )
        
        fig.update_layout(
            title_text="Reconstruction Method Comparison",
            showlegend=False
        )
        fig.show()
    
    return method_stats

# Run analysis
analysis_results = analyze_reconstruction_quality(reconstruction_results)


## Reprojection Error Analysis

In [ ]:
def calculate_reprojection_error(points_3d, pts1, pts2, P1, P2):
    """Calculate basic reprojection error for 3D points"""
    if len(points_3d) == 0:
        return float('inf'), float('inf')
    
    # Convert 3D points to homogeneous coordinates
    points_3d_homo = np.hstack([points_3d, np.ones((len(points_3d), 1))])
    
    # Project 3D points back to both images
    proj1 = P1 @ points_3d_homo.T
    proj2 = P2 @ points_3d_homo.T
    
    # Convert from homogeneous to 2D
    proj1_2d = (proj1[:2] / proj1[2]).T
    proj2_2d = (proj2[:2] / proj2[2]).T
    
    # Calculate reprojection errors
    error1 = np.linalg.norm(pts1[:len(points_3d)] - proj1_2d, axis=1)
    error2 = np.linalg.norm(pts2[:len(points_3d)] - proj2_2d, axis=1)
    
    return np.mean(error1), np.mean(error2)

def analyze_reprojection_errors(reconstruction_results):
    """Basic quantitative analysis using reprojection error"""
    
    print("\n=== Reprojection Error Analysis ===\n")
    
    for method, results in reconstruction_results.items():
        if not results:
            continue
            
        print(f"{method}:")
        method_errors = []
        
        for result in results:
            # Get data for this reconstruction
            points_3d = result['points_3d']
            P1, P2 = result['pose']
            pair = result['pair']
            
            # Load frames and get matching points
            frame1, frame2 = pair
            img1 = load_frame(frame1)
            img2 = load_frame(frame2)
            
            if img1 is None or img2 is None:
                continue
                
            # Get feature matches for this pair (simplified - using first N points)
            try:
                i1 = EXTRACTED_FRAME_INDICES.index(frame1)
                i2 = EXTRACTED_FRAME_INDICES.index(frame2)
                
                kps1 = kps_dict[method][i1]
                desc1 = desc_dict[method][i1]
                kps2 = kps_dict[method][i2]
                desc2 = desc_dict[method][i2]
                
                # Get matches
                if method == "ORB" or method == "AKAZE":
                    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                    matches = bf.match(desc1, desc2)
                elif method == "SIFT":
                    bf = cv2.BFMatcher(cv2.NORM_L2)
                    raw_matches = bf.knnMatch(desc1, desc2, k=2)
                    matches = [m for m, n in raw_matches if m.distance < 0.75 * n.distance]
                
                # Get point correspondences
                pts1 = np.float32([kps1[m.queryIdx].pt for m in matches[:len(points_3d)]])
                pts2 = np.float32([kps2[m.trainIdx].pt for m in matches[:len(points_3d)]])
                
                # Calculate reprojection errors
                error1, error2 = calculate_reprojection_error(points_3d, pts1, pts2, P1, P2)
                avg_error = (error1 + error2) / 2
                method_errors.append(avg_error)
                
                print(f"  Pair ({frame1}, {frame2}): {avg_error:.2f} pixels")
                
            except (ValueError, IndexError):
                continue
        
        if method_errors:
            overall_avg = np.mean(method_errors)
            print(f"  Average reprojection error: {overall_avg:.2f} pixels")
        else:
            print("  No valid error measurements")
        print()

# Run reprojection error analysis
analyze_reprojection_errors(reconstruction_results)